In [1]:
!date

Sun Dec  6 21:09:57 PST 2020


# All clusters DE

In [1]:
import pandas as pd
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from scipy import stats
import os
import anndata
import matplotlib.transforms as transforms
import matplotlib.patches as mpatches
import glob
import sys
def nd(arr):
    return np.asarray(arr).reshape(-1)

import sys
sys.path.append('/home/sina/projects/mop/BYVSTZP_2020/dexpress')
from dexpress import dexpress, utils, plot

sys.path.append('/home/sina/projects/mop/BYVSTZP_2020/trackfig')
from trackfig.utils import get_notebook_name
from trackfig.trackfig import trackfig 

TRACKFIG = "/home/sina/projects/mop/BYVSTZP_2020/trackfig.txt"
NB = get_notebook_name()


def yex(ax):
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]
    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [2]:
cluster_cmap = {
"Astro": (0.38823529411764707, 0.4745098039215686,  0.2235294117647059 ),  # 637939,
"Endo" : (0.5490196078431373,  0.6352941176470588,  0.3215686274509804 ),  # 8ca252,
"SMC"  : (0.7098039215686275,  0.8117647058823529,  0.4196078431372549 ),  # b5cf6b,
"VLMC" : (0.807843137254902,   0.8588235294117647,  0.611764705882353  ),  # cedb9c,
"Low Quality" : (0,0,0),
"L2/3 IT" : (0.9921568627450981,  0.6823529411764706,  0.4196078431372549  ),  # fdae6b
"L5 PT" : (0.9921568627450981,  0.8156862745098039,  0.6352941176470588  ),  # fdd0a2
"L5 IT" : (0.5176470588235295,  0.23529411764705882, 0.2235294117647059 ),  # 843c39
"L5/6 NP": "#D43F3A",
"L6 CT" : (0.8392156862745098,  0.3803921568627451,  0.4196078431372549 ),  # d6616b
"L6 IT" : (0.9058823529411765,  0.5882352941176471,  0.611764705882353  ),  # e7969c
"L6b" : (1.0,                 0.4980392156862745,  0.054901960784313725),  # ff7f0e
"L6 IT Car3" : (1.0,                 0.7333333333333333,  0.47058823529411764 ),  # ffbb78
"Lamp5" : (0.19215686274509805, 0.5098039215686274,  0.7411764705882353  ),  # 3182bd # blues
"Sncg" : (0.4196078431372549,  0.6823529411764706,  0.8392156862745098  ),  # 6baed6
"Vip" : (0.6196078431372549,  0.792156862745098,   0.8823529411764706  ),  # 9ecae1
"Sst" : (0.7764705882352941,  0.8588235294117647,  0.9372549019607843  ),  # c6dbef
"Pvalb":(0.7372549019607844,  0.7411764705882353,  0.8627450980392157  ),  # bcbddc
}

In [3]:
num_TSNE = 2
state = 42
metric = "euclidean"
n_neighbors = 30
num_PCA = 25
num_NCA = 10

# Filtering criteria
cell_threshold = 250
disp_threshold = 0.001

mito_criteria = 10

n_top_genes = 5000

n_bins = 20

flavor="seurat"

scale_clip = 10

In [4]:
import ast

In [5]:
gene = anndata.read_h5ad("../../data/notebook/revision/gene.h5ad")
isoform = anndata.read_h5ad("../../data/notebook/revision/isoform.h5ad")

In [6]:
gene_id = gene.var["gene_id"].values
gene_names = gene.var["gene_name"].values.astype(str)

# Restrict to genes with more than one isoform

In [7]:
gene = gene[:,gene.var["num_isoforms"]>1]

In [8]:
%%time
transcripts = []
l = gene.var.txn_list.values
for sublist in l:
    sublist = ast.literal_eval(sublist)
    for item in sublist:
        transcripts.append(item)

CPU times: user 223 ms, sys: 16.9 ms, total: 240 ms
Wall time: 239 ms


In [9]:
isoform = isoform[:,isoform.var["transcript_id"].isin(transcripts)]

In [10]:
print(gene)
print(isoform)

View of AnnData object with n_obs × n_vars = 6160 × 11891
    obs: 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size', 'cell_id', 'subclass_id', 'class_id', 'cell_counts', 'percent_mito', 'n_genes', 'pass_count_filter', 'leiden', 'louvain'
    var: 'transcript_id', 'gene_id', 'gene_name', 'transcript_name', 'length', 'txn_list', 'num_isoforms', 'gene_counts', 'dispersion', 'pass_disp_filter', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'leiden', 'louvain', 'neighbors'
    obsm: 'X_nca', 'X_nca_tsne', 'X_pca', 'X_pca_tsne'
    layers: 'X', 'log1p', 'norm'
    obsp: 'connectivities', 'distances'
View of AnnData object with n_obs × n_vars = 6160 × 61873
    obs: 'cluster_id', 'cluster_label', 'subclass_label', 'class_label', 'cluster_color', 'size', 'cell_id', 'subclass_id', 'class_id', 'cell_counts', 'leiden', 'louvain'
    var: 'transcript_id', 'gene_id', 'gene_name', 'transcript_name', 'length', 'gene_counts', 'high

In [11]:
isoform = isoform[isoform.obs.sort_values(["cluster_label", "cell_id"]).index]
gene = gene[gene.obs.sort_values(["cluster_label", "cell_id"]).index]

In [12]:
False in (gene.obs.cluster_label == isoform.obs.cluster_label)

False

In [13]:
isoform.obs

,cluster_id,cluster_label,subclass_label,class_label,cluster_color,size,cell_id,subclass_id,class_id,cell_counts,leiden,louvain
112,58,Astro Aqp4,Astro,Non-Neuronal,#7C2D1F,10,LS-15501_S47_E1-50,0,3,985306.8750,7,7
1073,58,Astro Aqp4,Astro,Non-Neuronal,#7C2D1F,10,SM-D9D87_S28_E1-50,0,3,982938.0625,7,7
4464,58,Astro Aqp4,Astro,Non-Neuronal,#7C2D1F,10,SM-DD449_S01_E1-50,0,3,999301.3125,14,4
4991,58,Astro Aqp4,Astro,Non-Neuronal,#7C2D1F,10,SM-GE65I_S103_E1-50,0,3,990256.1875,7,7
4992,58,Astro Aqp4,Astro,Non-Neuronal,#7C2D1F,10,SM-GE65I_S104_E1-50,0,3,987743.6875,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...
6471,6,Vip Serpinf1,Vip,GABAergic,#A720FF,50,SM-GE935_S268_E1-50,17,0,997492.0625,12,12
6532,6,Vip Serpinf1,Vip,GABAergic,#A720FF,50,SM-GE935_S337_E1-50,17,0,998825.7500,12,12
6537,6,Vip Serpinf1,Vip,GABAergic,#A720FF,50,SM-GE935_S342_E1-50,17,0,998837.6875,5,12
6545,6,Vip Serpinf1,Vip,GABAergic,#A720FF,50,SM-GE935_S350_E1-50,17,0,998223.1250,12,12


# determine the isoforms

In [25]:
def violinplot(data, ax, **kwd): 
    xticklabels = kwd.get("xticklabels", [])
    xticks = kwd.get("xticks", [])
    color = kwd.get("color", "#D43F3A")
    
    if  len(xticks)==0: xticks = np.arange(len(data))+1;
    if  len(xticklabels)==0: xticklabels = np.arange(len(data))+1;
    assert(len(xticks) == len(xticklabels))
        
    violins = ax.violinplot(data, positions=xticks, showmeans=False, showmedians=False, showextrema=False)
    
    for vidx, v in enumerate(violins['bodies']):
        v.set_facecolor(color)
        v.set_edgecolor('black')
        v.set_alpha(1)
    
    for didx, d in enumerate(data):
        x = xticks[didx]
        xx = np.random.normal(x, 0.04, size=len(d))
        
        # actual points
        ax.scatter(xx, d, s = 2, color="grey")
        
        # mean and error bars
        mean = np.mean(d)
        stdev = np.sqrt(np.var(d))
        ax.scatter(x, mean,color="black")    
        ax.vlines(x, mean - stdev, mean+stdev, color='black', linestyle='-', lw=2)
    
    return ax


# do for all clusters with a certain number of cells per cluster

In [19]:
subclasses = np.sort(isoform.obs.subclass_label.unique())

subclasses = np.setdiff1d(subclasses, ["L5 IT", "Low Quality"]) 

In [45]:
isoform.obs.groupby("subclass_label")["cluster_label"].value_counts()

subclass_label  cluster_label    
Astro           Astro Aqp4             9
Endo            Endo Slc38a5           7
L2/3 IT         L2/3 IT S100a6       231
                L2/3 IT Macc1        128
                L2/3 IT Cdh13 Sla    123
                                    ... 
Vip             Vip Htr1f             48
                Vip Serpinf1          48
                Vip Gpc3              37
                Vip Chat_1            33
                Vip Lmo1              25
Name: cluster_label, Length: 62, dtype: int64

In [46]:
%%time
n_cells = 5

de_clusters = []
de_genes = []
de_isoforms = []
for cidx, c in enumerate(subclasses):
    print(f"{cidx+1} of {len(subclasses)}: {c}")
    tmp_isoform = isoform[isoform.obs.eval(f"subclass_label == '{c}'")].copy()
    tmp_gene    = gene[gene.obs.eval(f"subclass_label == '{c}'")].copy()

    if tmp_isoform.shape[0] >= n_cells: # cluster must have at least 20 cells
        if tmp_isoform.obs.cluster_label.nunique()>1:
            de_clusters.append(c)

            ####### Genes
            mat           = tmp_gene.layers["log1p"].todense()
            components    = tmp_gene.obs.cell_id.values
            features      = tmp_gene.var.gene_name.values
            assignments   = tmp_gene.obs.cluster_label.values

            # parameters
            unique        = np.unique(assignments)
            nan_cutoff    = 0.9 # of elements in cluster
            corr_method   = "bonferroni"

            p_raw, stat, es, nfeat = dexpress.dexpress(mat, components, features, assignments, nan_cutoff=nan_cutoff)
            p_corr = utils.correct_pvals(p_raw)#, nfeat, corr_method)

            s = stat

            markers_gene = dexpress.make_table(assignments, features, p_raw, p_corr, es)

            # convert the 0 pvalues to the smallest possible float
            markers_gene["p_corr"][markers_gene.eval("p_corr == 0").values] = sys.float_info.min
            markers_gene["n_isoforms"] = markers_gene.name.map(gene.var.num_isoforms)
            de_genes.append(markers_gene)



            ######### Isoforms
            mat           = tmp_isoform.layers["log1p"].todense()
            components    = tmp_isoform.obs.cell_id.values
            features      = tmp_isoform.var.transcript_name.values
            assignments   = tmp_isoform.obs.cluster_label.values

            # parameters
            unique        = np.unique(assignments)
            nan_cutoff    = 0.9 # of elements in cluster
            corr_method   = "bonferroni"


            p_raw, stat, es, nfeat = dexpress.dexpress(mat, components, features, assignments, nan_cutoff=nan_cutoff)
            p_corr = utils.correct_pvals(p_raw)#, nfeat, corr_method)

            s = stat

            markers_isoform = dexpress.make_table(assignments, features, p_raw, p_corr, es)

            # convert the 0 pvalues to the smallest possible float
            markers_isoform["p_corr"][markers_isoform.eval("p_corr == 0").values] = sys.float_info.min
            de_isoforms.append(markers_isoform)

1 of 16: Astro
2 of 16: Endo
3 of 16: L2/3 IT


10-Dec-20 10:47:34 - 1 of 3 assignments: L2/3 IT Cdh13 Sla
10-Dec-20 10:47:34 - 2 of 3 assignments: L2/3 IT Macc1
10-Dec-20 10:47:35 - 3 of 3 assignments: L2/3 IT S100a6
10-Dec-20 10:47:35 - 1 of 3 assignments: L2/3 IT Cdh13 Sla
10-Dec-20 10:47:35 - 2 of 3 assignments: L2/3 IT Macc1
10-Dec-20 10:47:35 - 3 of 3 assignments: L2/3 IT S100a6


4 of 16: L5 PT
5 of 16: L5/6 NP


10-Dec-20 10:47:36 - 1 of 2 assignments: L5 NP Slc17a8
10-Dec-20 10:47:36 - 2 of 2 assignments: L6 NP Trh
10-Dec-20 10:47:36 - 1 of 2 assignments: L5 NP Slc17a8
10-Dec-20 10:47:36 - 2 of 2 assignments: L6 NP Trh


6 of 16: L6 CT


...

14 of 16: Sst


10-Dec-20 10:47:50 - 1 of 7 assignments: Sst C1ql3
10-Dec-20 10:47:50 - 2 of 7 assignments: Sst Chodl
10-Dec-20 10:47:50 - 3 of 7 assignments: Sst Crhr2_1
10-Dec-20 10:47:50 - 4 of 7 assignments: Sst Crhr2_2
10-Dec-20 10:47:50 - 5 of 7 assignments: Sst Etv1
10-Dec-20 10:47:50 - 6 of 7 assignments: Sst Myh8_1
10-Dec-20 10:47:50 - 7 of 7 assignments: Sst Myh8_2
10-Dec-20 10:47:50 - 1 of 7 assignments: Sst C1ql3
10-Dec-20 10:47:50 - 2 of 7 assignments: Sst Chodl
10-Dec-20 10:47:50 - 3 of 7 assignments: Sst Crhr2_1
10-Dec-20 10:47:50 - 4 of 7 assignments: Sst Crhr2_2
10-Dec-20 10:47:50 - 5 of 7 assignments: Sst Etv1
10-Dec-20 10:47:50 - 6 of 7 assignments: Sst Myh8_1
10-Dec-20 10:47:50 - 7 of 7 assignments: Sst Myh8_2


15 of 16: VLMC
16 of 16: Vip


10-Dec-20 10:47:52 - 1 of 10 assignments: Vip C1ql1
10-Dec-20 10:47:52 - 2 of 10 assignments: Vip Chat_1
10-Dec-20 10:47:52 - 3 of 10 assignments: Vip Chat_2
10-Dec-20 10:47:52 - 4 of 10 assignments: Vip Chat_3
10-Dec-20 10:47:52 - 5 of 10 assignments: Vip Crispld2
10-Dec-20 10:47:52 - 6 of 10 assignments: Vip Gpc3
10-Dec-20 10:47:52 - 7 of 10 assignments: Vip Htr1f
10-Dec-20 10:47:52 - 8 of 10 assignments: Vip Igfbp6
10-Dec-20 10:47:52 - 9 of 10 assignments: Vip Lmo1
10-Dec-20 10:47:52 - 10 of 10 assignments: Vip Serpinf1
10-Dec-20 10:47:53 - 1 of 10 assignments: Vip C1ql1
10-Dec-20 10:47:53 - 2 of 10 assignments: Vip Chat_1
10-Dec-20 10:47:53 - 3 of 10 assignments: Vip Chat_2
10-Dec-20 10:47:53 - 4 of 10 assignments: Vip Chat_3
10-Dec-20 10:47:53 - 5 of 10 assignments: Vip Crispld2
10-Dec-20 10:47:53 - 6 of 10 assignments: Vip Gpc3
10-Dec-20 10:47:53 - 7 of 10 assignments: Vip Htr1f
10-Dec-20 10:47:53 - 8 of 10 assignments: Vip Igfbp6
10-Dec-20 10:47:54 - 9 of 10 assignments: Vip Lmo

CPU times: user 18.9 s, sys: 2.35 s, total: 21.2 s
Wall time: 21.2 s


In [47]:
markers_gene = pd.concat(de_genes)

markers_isoform = pd.concat(de_isoforms)

markers_isoform["index"].nunique()

In [55]:
alpha = 0.01
markers_gene = markers_gene.query(f"p_corr < {alpha}")
markers_isoform = markers_isoform.query(f"p_corr < {alpha}")

In [62]:
# write isoform_only
markers_isoform.to_csv("../../tables/unordered/all_clusters_DE_isoform_only.csv")

In [63]:
relevant_genes = markers_gene.p_corr < alpha

markers_gene["index_name"] = markers_gene["index"] + "_" + markers_gene.name.apply(lambda x: "".join(x.split("_")[:-1]))
markers_isoform["index_name"] = markers_isoform["index"] + "_" + markers_isoform.name.apply(lambda x: "-".join(x.split("-")[:-1]))

setdiff = np.setdiff1d(markers_isoform["index_name"].values, markers_gene[relevant_genes]["index_name"].values)

/home/sina/bin/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [64]:
markers_isoform = markers_isoform[markers_isoform["index_name"].isin(setdiff)].sort_values(["es", "p_corr"])

markers_isoform = markers_isoform.query(f"p_corr < {alpha}")
markers_isoform.sort_values("es")[::-1][:20]

,index,name,p_raw,p_corr,es,index_name
98111,L6b Kcnip1_2,Nrn1l-201_ENSMUST00000060167,1.20735e-06,0.00663437,4.01019,L6b Kcnip1_2_Nrn1l
255138,Pvalb Vipr2,Snap25-202_ENSMUST00000110098,1.54105e-17,5.41525e-14,3.57294,Pvalb Vipr2_Snap25
116269,Sncg Slc17a8,Cacnb3-202_ENSMUST00000109150,1.91314e-08,0.000102984,2.95097,Sncg Slc17a8_Cacnb3
116507,L6b Kcnip1_2,Krt80-205_ENSMUST00000230909,6.4883e-07,0.00356532,2.8745,L6b Kcnip1_2_Krt80
118796,Sncg Slc17a8,Atp5o-206_ENSMUST00000155452,6.01467e-09,3.2377e-05,2.69355,Sncg Slc17a8_Atp5o
75956,L6b Kcnip1_2,St6galnac5-201_ENSMUST00000044278,8.52668e-07,0.00468541,2.68449,L6b Kcnip1_2_St6galnac5
393690,Sst Myh8_2,Dync1i1-206_ENSMUST00000203520,4.0469e-21,1.08902e-17,2.67499,Sst Myh8_2_Dync1i1
84594,Sncg Slc17a8,Impdh1-209_ENSMUST00000162242,9.37485e-08,0.000504648,2.65715,Sncg Slc17a8_Impdh1
287305,L6 CT Nxph2_2,Ntm-209_ENSMUST00000151977,2.40782e-07,0.000721624,2.54155,L6 CT Nxph2_2_Ntm
54996,L6 IT Sulf_1,Rbfox1-208_ENSMUST00000231194,1.804e-25,1.10567e-21,2.36057,L6 IT Sulf_1_Rbfox1


In [68]:
markers_isoform.groupby("index")["name"].nunique()

index
L2/3 IT Cdh13 Sla     49
L2/3 IT Macc1         30
L2/3 IT S100a6        26
L5 NP Slc17a8         28
L6 CT Gpr139          47
L6 CT Grp_1           19
L6 CT Grp_2           20
L6 CT Nxph2_1         14
L6 CT Nxph2_2         13
L6 IT Sulf_1          84
L6 IT Sulf_2          29
L6 IT Sulf_3          30
L6 NP Trh             24
L6b Kcnip1_1          40
L6b Kcnip1_2          47
L6b Ror1_1            54
L6b Ror1_2            32
L6b Rprm              53
L6b Shisa6            45
Lamp5 Egln3_1          9
Lamp5 Egln3_2         14
Lamp5 Lhx6            17
Lamp5 Pdlim5           6
Lamp5 Slc35d3         49
Pvalb Calb1           27
Pvalb Gpr149          45
Pvalb Reln            12
Pvalb Th              18
Pvalb Vipr2           25
Sncg                  15
Sncg Slc17a8          67
Sst C1ql3             53
Sst Chodl             32
Sst Crhr2_1           16
Sst Crhr2_2           20
Sst Etv1              19
Sst Myh8_1             7
Sst Myh8_2            10
Vip C1ql1             19
Vip Chat_1         